<a href="https://colab.research.google.com/github/SDS-AAU/SDS-master/blob/master/M1/notebooks/exercises/UML_application_R_ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Sorry guys, this takes some time....
install.packages("FactoMineR")
install.packages("factoextra")

In [ ]:
library(tidyverse) # metapackage with lots of helpful functions
library(magrittr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract



# Read the data
Lets load some data!  You will find a dataset on Gert Hofstede's ["6-D model of national culture""](https://geerthofstede.com/culture-geert-hofstede-gert-jan-hofstede/6d-model-of-national-culture/). This popular measures of country-level culture in (by now) 6 dimensions became very popular in sociology, economics, and management science to explain cross-cultural interaction as well as frictions. a exhaustive documentation of the 2013 dataset can be found [here](https://geerthofstede.com/wp-content/uploads/2016/07/Manual-VSM-2013.pdf).

In [ ]:
data <- read_csv2("https://www.dropbox.com/s/6can8ofrh1mqukh/vsm13.csv?dl=1", na = "#NULL!")

Using ',' as decimal and '.' as grouping mark. Use read_delim() for more control.
Parsed with column specification:
cols(
  ctr = col_character(),
  country = col_character(),
  pdi = col_double(),
  idv = col_double(),
  mas = col_double(),
  uai = col_double(),
  ltowvs = col_double(),
  ivr = col_double()
)


In [ ]:
data %>% head()

ctr,country,pdi,idv,mas,uai,ltowvs,ivr
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AFE,Africa East,64,27,41,52,32,40
AFW,Africa West,77,20,46,54,9,78
ALB,Albania,NA,NA,NA,NA,61,15
ALG,Algeria,NA,NA,NA,NA,26,32
AND,Andorra,NA,NA,NA,NA,NA,65
ARA,Arab countries,80,38,53,68,23,34


In [ ]:
data %>% glimpse()

Observations: 111
Variables: 8
$ ctr     <chr> "AFE", "AFW", "ALB", "ALG", "AND", "ARA", "ARG", "ARM", "AUL"…
$ country <chr> "Africa East", "Africa West", "Albania", "Algeria", "Andorra"…
$ pdi     <dbl> 64, 77, NA, NA, NA, 80, 49, NA, 38, 11, NA, 80, NA, 65, 67, 6…
$ idv     <dbl> 27, 20, NA, NA, NA, 38, 46, NA, 90, 55, NA, 20, NA, 75, 72, 7…
$ mas     <dbl> 41, 46, NA, NA, NA, 53, 56, NA, 61, 79, NA, 55, NA, 54, 60, 4…
$ uai     <dbl> 52, 54, NA, NA, NA, 68, 86, NA, 51, 70, NA, 60, NA, 94, 93, 9…
$ ltowvs  <dbl> 32, 9, 61, 26, NA, 23, 20, 61, 21, 60, 61, 47, 81, 82, NA, NA…
$ ivr     <dbl> 40, 78, 15, 32, 65, 34, 62, NA, 71, 63, 22, 20, 15, 57, NA, N…


* **`pdi:`**  The power distance index is defined as "the extent to which the less powerful members of organizations and institutions (like the family) accept and expect that power is distributed unequally."In this dimension, inequality and power is perceived from the followers, or the lower level. A higher degree of the Index indicates that hierarchy is clearly established and executed in society, without doubt or reason. A lower degree of the Index signifies that people question authority and attempt to distribute power.
* **`idv:`**  This index explores the "degree to which people in a society are integrated into groups."" Individualistic societies have loose ties that often only relates an individual to his/her immediate family. They emphasize the "I" versus the "we". Its counterpart, collectivism, describes a society in which tightly-integrated relationships tie extended families and others into in-groups. These in-groups are laced with undoubted loyalty and support each other when a conflict arises with another in-group.
* **`mas:`**  In this dimension, masculinity is defined as "a preference in society for achievement, heroism, assertiveness and material rewards for success."" Its counterpart represents "a preference for cooperation, modesty, caring for the weak and quality of life." Women in the respective societies tend to display different values. In feminine societies, they share modest and caring views equally with men. In more masculine societies, women are somewhat assertive and competitive, but notably less than men. In other words, they still recognize a gap between male and female values. This dimension is frequently viewed as taboo in highly masculine societies.
* **`uai:`**  The uncertainty avoidance index is defined as "a society's tolerance for ambiguity," in which people embrace or avert an event of something unexpected, unknown, or away from the status quo. Societies that score a high degree in this index opt for stiff codes of behavior, guidelines, laws, and generally rely on absolute truth, or the belief that one lone truth dictates everything and people know what it is. A lower degree in this index shows more acceptance of differing thoughts or ideas. Society tends to impose fewer regulations, ambiguity is more accustomed to, and the environment is more free-flowing.
* **`ltowvs:`** This dimension associates the connection of the past with the current and future actions/challenges. A lower degree of this index (short-term) indicates that traditions are honored and kept, while steadfastness is valued. Societies with a high degree in this index (long-term) views adaptation and circumstantial, pragmatic problem-solving as a necessity. A poor country that is short-term oriented usually has little to no economic development, while long-term oriented countries continue to develop to a point. 
* **`ivr:`**  This dimension is essentially a measure of happiness; whether or not simple joys are fulfilled. Indulgence is defined as "a society that allows relatively free gratification of basic and natural human desires related to enjoying life and having fun." Its counterpart is defined as "a society that controls gratification of needs and regulates it by means of strict social norms. Indulgent societies believe themselves to be in control of their own life and emotions; restrained societies believe other factors dictate their life and emotions.

Ok, looks interesting. Let's do the fololwing:

0. The data is not perfect. So some small upfront-munging is necessary.
1. Gert Hofstede claims this dimensions to emasure orthogonal features of culture. That raises the question if they reasy measure different constructs. To find out, lets execute a PCA on them. How do the dimensions load? And how do countries score? Illustrate and visualize the results.
2. Can we form meaningful "cultural clusters" among countries?
3. Let's create a meaningful measure for "cultural distance" between countries. What do we see? Interpret.
4. (Advanced) Does bilateral "cultural distance" or the assignment to a "cultural cluster" help us to explain other interaction between countries we might be interested in, such as trade, migration etc.? Here you will need some skills from M1-1 & 2.

In [ ]:
# Load the packages we need
library(FactoMineR)
library(factoextra)

Some preprocessing upfront

In [ ]:
# First, let's get rid of NAs (we could impute them, but lets be lazy for now)
data %<>% drop_na()

# define rownames (for the visualization)
rownames(data) <- data %>% pull(***)

Lets do a kmeans clustering

In [ ]:
km <- kmeans(***, centers = ***)  
glimpse(km)

And visualize it...

In [ ]:
fviz_cluster(***)  

Can we summarize the cluster means to categorize clusters?

In [ ]:
data %>%
  bind_cols(cluster = ***) %>%
  select(***) %>%
  group_by(cluster) %>%
  mutate(n = n()) %>%
  summarise_all(funs(mean))